In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt

# Paths to train and test directories
train_dir = './cifar-10-img/train'
test_dir = './cifar-10-img/test'

# a. Load images with Keras from directory
train_dataset = image_dataset_from_directory(
    train_dir,
    label_mode='int',
    image_size=(32, 32),  # Resizing to 224x224 for VGG16
    batch_size=32,
    color_mode='rgb'
)

test_dataset = image_dataset_from_directory(
    test_dir,
    label_mode='int',
    image_size=(32, 32),  # Resizing to 224x224 for VGG16
    batch_size=32,
    color_mode='rgb'
)

# Normalize the images
def normalize_images(image, label):
    return image / 255.0, label

train_dataset = train_dataset.map(normalize_images)
test_dataset = test_dataset.map(normalize_images)

# a. Load a pre-trained model (VGG16), excluding the top classifier layer
base_model = VGG16(weights='./vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(32, 32, 3))

# b. Freeze parameters (weights) in model’s lower convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# c. Add custom classifier with several layers of trainable parameters
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)  # 10 classes for CIFAR-10

# Create the new model
model = Model(inputs=base_model.input, outputs=output)

# d. Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# d. Train classifier layers on training data
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset
)

# e. Fine-tune hyperparameters and unfreeze more layers if needed
for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Continue training with fine-tuning
fine_tune_history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=test_dataset
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# f. Plot the training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(fine_tune_history.history['accuracy'], label='Fine-tune Training Accuracy')
plt.plot(fine_tune_history.history['val_accuracy'], label='Fine-tune Validation Accuracy')

plt.title('Training and Validation Accuracy with Fine-Tuning')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Found 40079 files belonging to 10 classes.
Found 9921 files belonging to 10 classes.
Epoch 1/10
1253/1253 [==============================] - 208s 165ms/step - loss: 1.5881 - accuracy: 0.4376 - val_loss: 1.3240 - val_accuracy: 0.5383
Epoch 2/10
1253/1253 [==============================] - 181s 145ms/step - loss: 1.3831 - accuracy: 0.5177 - val_loss: 1.2519 - val_accuracy: 0.5673
Epoch 3/10
1253/1253 [==============================] - 176s 140ms/step - loss: 1.3169 - accuracy: 0.5425 - val_loss: 1.2200 - val_accuracy: 0.5774
Epoch 4/10
1253/1253 [==============================] - 178s 142ms/step - loss: 1.2780 - accuracy: 0.5536 - val_loss: 1.1939 - val_accuracy: 0.5859
Epoch 5/10
1253/1253 [==============================] - 176s 141ms/step - loss: 1.2518 - accuracy: 0.5671 - val_loss: 1.1907 - val_accuracy: 0.5842
Epoch 6/10
1253/1253 [==============================] - 181s 144ms/step - loss: 1.2308 - accuracy: 0.5729 - val_loss: 1.1814 - val_accuracy: 0.5910
Epoch 7/10
1253/1253 [=====